# Import the Necessary Libraries

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torchsummary import summary # pip install torch-summary
from tqdm import tqdm

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cpu


In [3]:
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

# Download the CIFAR 10 Dataset

In [4]:
# Get the training data and create a transform to ensure the data is made up of tensors
# why are we not normalizing the data between 0 and 1?
# Because it benefits,like centering the data around zero and ensuring that the data has a similar scale across different features (channels)
transform = transforms.Compose([transforms.ToTensor()])
train_data = torchvision.datasets.CIFAR10(root='./data', train=True, download = True, transform=transform)

# load the training data and transform it
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)

# Get the test data
test_data = torchvision.datasets.CIFAR10(root='./data', train=False, download = True, transform=transform)

# Now load the test data with a batch size of 32: Do we need to shuffle the test data?

test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2)

100%|██████████| 170498071/170498071 [00:02<00:00, 67234102.64it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


### Do we need to shuffle the test data?
No, because for the test loader we have to ensure that the data remains in the same order for evaluation, allowing for easier comparison of results and ensuring that each sample is tested exactly once


# Building the Model

In [5]:
class MLP(torch.nn.Module):

    def __init__(self):
        super(MLP, self).__init__()
        # Create instance variables for the layers you will pass in the forward method
        # write your code below
        self.fclayer1 = torch.nn.Linear(32 * 32 * 3, 200) # The first layer that output 200
        self.fclayer2 = torch.nn.Linear(200, 150)
        self.fclayer3 = torch.nn.Linear(150, 10)

    def forward(self, x):
        # complete the forward method
        x = x.view(-1, 32 * 32 * 3) # The flattened input tensor
        x = torch.nn.functional.relu(self.fclayer1(x))
        x = torch.nn.functional.relu(self.fclayer2(x))
        x = self.fclayer3(x)

        # The output to the softmax
        output = torch.nn.functional.softmax(x, dim=1)

        return output

mlp_model = MLP().to(device)
summary(mlp_model, (3, 32, 32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 200]         614,600
            Linear-2                  [-1, 150]          30,150
            Linear-3                   [-1, 10]           1,510
Total params: 646,260
Trainable params: 646,260
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.00
Params size (MB): 2.47
Estimated Total Size (MB): 2.48
----------------------------------------------------------------


# Set the Loss Function and Optimizer

In [6]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mlp_model.parameters(), lr=0.0005)

# Write the Train Function

In [7]:
def train(dataloader, model, criterion, optimizer):
    model.train()
    batch_bar = tqdm(total=len(dataloader), position=0, leave=False, dynamic_ncols=True, desc='Train')
    total_loss = 0
    total_accuracy = 0

    for i, (imgs, labels) in enumerate(dataloader):
        # write your code below for the train function
        # Move data to device which is cpu
        imgs, labels = imgs.to(device), labels.to(device)

        # Forward pass
        outputs = model(imgs)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Compute accuracy
        _, predicted = torch.max(outputs, 1)
        accuracy = (predicted == labels).sum().item()

        total_loss += loss.item() * imgs.size(0)
        total_accuracy += accuracy
        # total_samples += labels.size(0)

        # update the progress bar
        batch_bar.set_postfix(loss=loss.item(), accuracy=100 * accuracy / len(labels))
        batch_bar.update()
    batch_bar.close()
    return total_loss/len(dataloader), 100*total_accuracy/(len(dataloader)*32)

## Write the Evaluate Function to get the Test Accuracy

In [8]:
def evaluate(dataloader, model):
    model.eval()
    # write your code for the evaluate function
    accuracy = 0
    total = 0

    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)

            # Get predicted labels
            _, predicted = torch.max(outputs.data, 1)

            # Total number of samples
            total += labels.size(0)

            # Number of correctly classified samples
            accuracy += (predicted == labels).sum().item()

    # Calculate test accuracy
    test_accuracy = 100 * accuracy / total
    return test_accuracy


# Train the MLP

In [9]:
for epoch in range(0, 10):
    train_loss, train_acc = train(train_dataloader, mlp_model, criterion, optimizer)
    print(f"Epoch {epoch+1}/{10} - Train Loss: {train_loss:.4f} - Train Accuracy: {train_acc:.4f}")

Epoch 1/10 - Train Loss: 68.8879 - Train Accuracy: 29.6825


Epoch 2/10 - Train Loss: 67.1928 - Train Accuracy: 35.2487


Epoch 3/10 - Train Loss: 66.5885 - Train Accuracy: 37.2101


Epoch 4/10 - Train Loss: 66.0638 - Train Accuracy: 39.0655


Epoch 5/10 - Train Loss: 65.6880 - Train Accuracy: 40.1412


Epoch 6/10 - Train Loss: 65.3530 - Train Accuracy: 41.2588


Epoch 7/10 - Train Loss: 65.1351 - Train Accuracy: 41.9346


Epoch 8/10 - Train Loss: 64.8502 - Train Accuracy: 42.6863


Epoch 9/10 - Train Loss: 64.5636 - Train Accuracy: 43.7680


Epoch 10/10 - Train Loss: 64.3608 - Train Accuracy: 44.4478


# Get Your Test Accuracy

In [10]:
test_acc = evaluate(test_dataloader, mlp_model)
print("The test accuracy is: ",test_acc)

The test accuracy is:  43.41
